***
some experiments with S1 collections
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils


__S1 Ground Range Detected (GRD)__

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD


***
__S1: different images can have different bands__
- mostly VV+VH or HH+HV
- property 'transmitterReceiverPolarisation': either ascending or descending
***

In [ ]:
#
# without extra filterBounds this can take a while;
# eg global from: 2019-01-01 till: 2019-12-31 there are 395743 S1 images in the collection
#
eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                     .filter(ee.Filter.date(ee.Date('2019-01-01'), ee.Date('2020-01-01')))
                     .filterBounds(geeutils.hoogeheydepoint.buffer(1000000)))


size_all     = (eeimagecollection.size().getInfo())
size_none    = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_VV_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_VH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HV_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
size_VVVH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HVHH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
size_VVVHHVHH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
                        

print(geeutils.szimagecollectioninfo(eeimagecollection))
print(f"all        : {size_all:15d}")
print(f"none       : {size_none:15d}")
print(f"VV only    : {size_VV_only:15d}   {size_VV_only/size_all:5.1%}")
print(f"VH only    : {size_VH_only:15d}   {size_VH_only/size_all:5.1%}")
print(f"HV only    : {size_HV_only:15d}   {size_HV_only/size_all:5.1%}")
print(f"HH only    : {size_HH_only:15d}   {size_HH_only/size_all:5.1%}")
print(f"VV+VH  only: {size_VVVH_only:15d}   {size_VVVH_only/size_all:5.1%}")
print(f"HV+HH  only: {size_HVHH_only:15d}   {size_HVHH_only/size_all:5.1%}")
print(f"VV+VH+HV+HH: {size_VVVHHVHH_only:15d}   {size_VVVHHVHH_only/size_all:5.1%}")

      

In [ ]:
#
# aggregate_histogram is faster but still...
#
eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                     .filter(ee.Filter.date(ee.Date('2019-01-01'), ee.Date('2020-01-01')))
                     .filterBounds(geeutils.hoogeheydepoint.buffer(1000000)))

print(geeutils.szimagecollectioninfo(eeimagecollection))
print("\ntransmitterReceiverPolarisation:\n", eeimagecollection.aggregate_histogram('transmitterReceiverPolarisation').getInfo())
print("\norbitProperties_pass:\n",            eeimagecollection.aggregate_histogram('orbitProperties_pass').getInfo())
print("\nrelativeOrbitNumber_start:\n",       eeimagecollection.aggregate_histogram('relativeOrbitNumber_start').getInfo())

***
__beware for decibels__

two collections:
- 'COPERNICUS/S1_GRD' : decibel (https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD)
- 'COPERNICUS/S1_GRD_FLOAT' : natural - but not to be found in documentation
- VV & VH can be converted, but whateved happened to 'angle' I cannot find out.
***

In [ ]:
eepoint           = geeutils.bobspoint
eeregion          = geeutils.squareareaboundsroi(eepoint, 1000) # 2km x 2km
eedatefrom        = geeutils.fleecycloudsday
eedatetill        = eedatefrom.advance(1, 'month')              # Beware for the "User memory limit exceeded."

eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(eeregion)
                     .filter(ee.Filter.date(eedatefrom, eedatetill)))
#
# collection info
#
#print(f"S1_GRD szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(eeimagecollection)}\n")
#
# estimate ranges
#
print(f"S1_GRD szestimatevaluesinfo: \n{geeutils.szestimatevaluesinfo(eeimagecollection)}\n")



eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                     .filterBounds(eeregion)
                     .filter(ee.Filter.date(eedatefrom, eedatetill)))
#
# collection info
#
#print(f"S1_GRD_FLOAT szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(eeimagecollection)}\n")
#
# estimate ranges
#
print(f"S1_GRD_FLOAT szestimatevaluesinfo: \n{geeutils.szestimatevaluesinfo(eeimagecollection)}\n")



eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                     .filterBounds(eeregion)
                     .filter(ee.Filter.date(eedatefrom, eedatetill)))

###################################################
# convert natural to decibel
###################################################
def tosigmadb(image):
    return (image
            .addBands(ee.Image(image.select('VV').log10().multiply(10.0)), overwrite=True)
            .addBands(ee.Image(image.select('VH').log10().multiply(10.0)), overwrite=True))

eeimagecollection = eeimagecollection.map(tosigmadb)

#
# collection info
#
#print(f"tosigmadb S1_GRD_FLOAT szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(eeimagecollection)}\n")
#
# estimate ranges
#
print(f"tosigmadb S1_GRD_FLOAT szestimatevaluesinfo: \n{geeutils.szestimatevaluesinfo(eeimagecollection)}\n")



eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(eeregion)
                     .filter(ee.Filter.date(eedatefrom, eedatetill)))

###################################################
# convert decibel to natural
###################################################
def fromsigmadb(image):
    return (image
            .addBands(ee.Image(ee.Image(10.0).pow(image.select('VV').divide(10.0))).select([0], ['VV']), overwrite=True)
            .addBands(ee.Image(ee.Image(10.0).pow(image.select('VH').divide(10.0))).select([0], ['VH']), overwrite=True))

eeimagecollection = eeimagecollection.map(fromsigmadb)

#
# collection info
#
#print(f"fromsigmadb S1_GRD szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(eeimagecollection)}\n")
#
# estimate ranges
#
print(f"fromsigmadb S1_GRD szestimatevaluesinfo: \n{geeutils.szestimatevaluesinfo(eeimagecollection)}\n")


***
__gamma 0 - sigma 0__

gamma0 = sigma0 / cos(t) &nbsp; &nbsp; &nbsp; &nbsp; (t = angle x pi / 180)\
=> 10 x log(gamma0) = 10 x log(sigma0) - 10 x log(cos(t))\
=> gamma0_db = sigma0_db - 10 x log(cos(t))

*log(average(variable)) != average(log(variable))*
***

In [ ]:

eepoint           = geeutils.bobspoint
eeregion          = geeutils.squareareaboundsroi(eepoint, 50) # 100m x 100m
eedatefrom        = geeutils.fleecycloudsday
eedatetill        = eedatefrom.advance(1, 'week')              # Beware for the "User memory limit exceeded."

eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(eeregion)
                     .filter(ee.Filter.date(eedatefrom, eedatetill)))

def undodbprojredodb(image, eeprojection):
    return (ee.Image(10.0).pow(image.divide(10.0))
            .reduceResolution(ee.Reducer.mean(), maxPixels=128*128)
            .reproject(eeprojection)
            .log10().multiply(10.0)
            .rename(image.bandNames()) # gotcha: eeimagecollection 2 bands: no problem, 1 band: name becomes 'constant': "The output bands are named for the longer of the two inputs, or if they're equal in length, in image1's order."
            .copyProperties(image)
            .copyProperties(image, ['system:id', 'system:time_start']))

def projasis(image, eeprojection):
    return (image
            .reduceResolution(ee.Reducer.mean(), maxPixels=128*128)
            .reproject(eeprojection))

vvcollection = eeimagecollection.select(['VV'])
print(f"szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(vvcollection)}\n")
print(f"szestimatevaluesinfo: \n{geeutils.szestimatevaluesinfo(vvcollection)}\n")
#
# downsampling: convert to natural - mean reducer - convert back to db
#
vvcollection_ok = vvcollection.map(lambda eeimage: undodbprojredodb(eeimage, vvcollection.first().projection().scale(64,64)))
#print(f"szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(vvcollection_ok)}\n")
print(f"szestimatevaluesinfo: \n{geeutils.szestimatevaluesinfo(vvcollection_ok)}\n")
#
# default downsampling: mean reducer
#
vvcollection_nok = vvcollection.map(lambda eeimage: projasis(eeimage, vvcollection.first().projection().scale(64,64)))
#print(f"szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(vvcollection_nok)}\n")
print(f"szestimatevaluesinfo: \n{geeutils.szestimatevaluesinfo(vvcollection_nok)}\n")
#
# note a lot of contrast getting lost by default downsampling
#
map = geemap.Map(height='600px')
map.centerObject(eepoint, 16)
map.addLayer(vvcollection,     {'min':-50, 'max':1}, 'vvcollection')
map.addLayer(vvcollection_ok,  {'min':-50, 'max':1}, 'vvcollection_ok')
map.addLayer(vvcollection_nok, {'min':-50, 'max':1}, 'vvcollection_nok')
map


***
__mind the gap__

- images can have weird footprints
- footprints have some overlap
- but data does not always 'fill' the footprint completely
- and although footprints ovelap, there can be data-gaps between images
***


In [ ]:

eepoint           = geeutils.hoogeheydepoint                  # 2-S1-ASC tiles on 2018-07-16 (hoogeheydeASCday)
eeregion          = geeutils.squareareaboundsroi(eepoint, 50) # 100m x 100m
eedatefrom        = geeutils.hoogeheydeASCday
eedatetill        = eedatefrom.advance(1, 'day')

eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(eeregion)
                     .filter(ee.Filter.date(eedatefrom, eedatetill))
                     .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                     .select(['VV']))
#
# expecting two images in this collection
#
print(geeutils.szimagecollectioninfo(eeimagecollection))

eeimage0   = ee.Image(eeimagecollection.toList(eeimagecollection.size()).get(0))
footprint0 = ee.Geometry(eeimage0.get('system:footprint'))
eeimage1   = ee.Image(eeimagecollection.toList(eeimagecollection.size()).get(1))
footprint1 = ee.Geometry(eeimage1.get('system:footprint'))

#
# note footprint vs data shifts
# note data gap between the two images
#
map = geemap.Map(height='600px')
map.centerObject(eepoint, 18)    #7) #18) # scale 7 for full images, scale 18 to view the gap
map.addLayer(eeimage0,     {'min':-50, 'max':1, 'palette':['#000000','#ff0000']}, 'eeimage0')
map.addLayer(eeimage1,     {'min':-50, 'max':1, 'palette':['#000000','#00ff00']}, 'eeimage1')
map.addLayer(geeutils.outlinegeometryimage(footprint0, 0, 3), {'palette':'#FF0000'}, 'footprint0')
map.addLayer(geeutils.outlinegeometryimage(footprint1, 0, 3), {'palette':'#00FF00'}, 'footprint1')
map


***
__angle band mysteries__
- angle band has values outside the system:footprint
- angle band has partially masked values - even inside footprint

***

In [ ]:
eepoint           = geeutils.hoogeheydepoint                  # 2-S1-ASC tiles on 2018-07-16 (hoogeheydeASCday)
eeregion          = geeutils.squareareaboundsroi(eepoint, 50) # 100m x 100m
eedatefrom        = geeutils.hoogeheydeASCday
eedatetill        = eedatefrom.advance(1, 'day')

eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filter(ee.Filter.eq('instrumentSwath', 'IW'))
                     .filter(ee.Filter.listContains('system:band_names','VV'))
                     .filter(ee.Filter.listContains('system:band_names','VH'))
                     .filter(ee.Filter.listContains('system:band_names','angle'))
                     .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                     .filterBounds(eeregion)
                     .filter(ee.Filter.date(eedatefrom, eedatetill)))

#
# check if we have an image (expect 2)
#
print(geeutils.szimagecollectioninfo(eeimagecollection))
print(geeutils.szestimatevaluesinfo(eeimagecollection))

#
# just pick one
#
image = ee.Image(eeimagecollection.first())

#
# note the weird tranformation matrix for the 'angle' band
#
print(geeutils.szprojectioninfo(image))

#
# note the 'angle' band showing pixels outside the system:footprint
#
vvimage = image.select(['VV'])
animage = image.select(['angle'])

#
# angle band has 'partially masked' pixels at the edges
# - we can leave things as they are and hope gee knows what it is doing
# - we can select only completely valid pixels (force partially masked to completely masked)
# - we can select all pixels available (force partially masked to unmasked)
#

#
# as-is : note that angle shows 'bleedover' 
#         https://groups.google.com/g/google-earth-engine-developers/c/3tVr0keZ9mQ/m/_vLE1v0NCAAJ)
#
zeimage = animage
#
# force 'partially masked' pixels to be 'unmasked' - consider everything 'valid'
#
#zeimage = animage.mask(animage.mask().gt(0))
#
# force 'partially masked' pixels to be 'masked' - consider valid only completely 'not masked'
#
#zeimage = animage.updateMask(animage.mask().eq(1)) # force 'partially masked' pixels to be 'unmasked'

map = geemap.Map()
map.centerObject(image, 8)
map.addLayer(zeimage, {'min':29,  'max':46, 'palette':['#0000ff','#ff0000']})
map.addLayer(vvimage, {'min':-50, 'max':1, 'opacity':0.7})
map.addLayer(geeutils.outlinegeometryimage(ee.Geometry(image.get('system:footprint')), 0, 3))

map